<a href="https://colab.research.google.com/github/prakul/MongoDB-AI-Resources/blob/main/image_search_modal_%2B_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymongo python-dotenv modal sentence-transformers numpy

# Step 1: Setup the environment


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

# Add an environment file to the notebook root directory called `.env` with params
# like MONGO_URI="xxx" in each line to load these envvars

MONGO_URI = os.environ["MONGO_URI"]

# Get your keys from modal.com/signup
os.environ["MODAL_TOKEN_ID"] = "XXX"
os.environ["MODAL_TOKEN_SECRET"] = "XXX"

In [ ]:
ATLAS_VECTOR_SEARCH_INDEX_NAME = "ImageSemanticSearch"
EMBEDDING_FIELD_NAME = "image_embedding_clip"

In [ ]:
import pymongo

mongodb_client = pymongo.MongoClient(MONGO_URI)
db = mongodb_client.sample_airbnb
collection = db.listingsAndReviews


In [ ]:

# [Optional] FOR DEBUG PURPOSE
# Test your connection and sample dataset setup by
# fetching one of the documents in the collection

#for doc in collection.find_one():
#    print(doc)


# Step 2: Setup embeddings generation function

In [ ]:
from modal.object import Object
import requests
from modal import Stub, Image, method

stub = Stub()

def download_models():
    # Caches the model inside the Modal image, so subsequent cold starts are faster.
    from sentence_transformers import SentenceTransformer
    SentenceTransformer('sentence-transformers/clip-ViT-B-32')

container_image = Image.debian_slim().pip_install("sentence-transformers").run_function(download_models)

@stub.cls(image=container_image)
class ModalEmbedding:
    def __enter__(self):
      from sentence_transformers import SentenceTransformer
      self.model = SentenceTransformer('sentence-transformers/clip-ViT-B-32')

    @method()
    def generate(self, image: Object):
      from PIL import Image
      if not 'picture_url' in image:
        return None
      try:
        image = Image.open(requests.get(image['picture_url'], stream=True).raw).convert('RGB')
      except Exception as exc:
        print(image['picture_url'], exc)
        return None
      img_emb = self.model.encode(image).tolist()
      return img_emb

In [ ]:
def generate_embedding(query):
    from sentence_transformers import SentenceTransformer, util
    import numpy
    model = SentenceTransformer('clip-ViT-B-32')
    # First, we encode the query (which can either be an image or a text string)
    query_emb = model.encode([query], show_progress_bar=False)
    return numpy.ndarray.tolist(query_emb)



In [ ]:

# [Optional] FOR DEBUG PURPOSE
# Test the generate embedding function
#img_url = 'https://a0.muscache.com/im/pictures/231120b6-e6e5-4514-93cd-53722ac67de1.jpg?aki_policy=large'
#query_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

#with stub.run():
#    print(len(ModalEmbedding().generate.remote(images[0])))


# Step 3: Create and store embeddings


In [ ]:
# Fetch the documents from MongoDB
docs = list(collection.find({'images.picture_url':{"$ne":None}}))



In [ ]:
docs1 = list(collection.find({'images':{"$exists": True}}))


In [ ]:
# Generate vector embeddings for the plot field in the dataset
with stub.run():
	images = [doc['images'] for doc in docs]
	embeddings = list(ModalEmbedding().generate.map(images))

In [ ]:
print(embeddings[0])

[-0.10067489743232727, 0.40229833126068115, -0.18212583661079407, 0.12891000509262085, 0.06888793408870697, -0.33011144399642944, -0.10319440066814423, 0.5468993782997131, 0.49641793966293335, 0.02749542146921158, 0.16331984102725983, -0.17224961519241333, 0.12043209373950958, -0.2894374132156372, 0.1871185302734375, -0.22775891423225403, -0.3175334632396698, 0.050206076353788376, 0.006322325207293034, 0.18621939420700073, 1.3562796115875244, 0.304808646440506, 0.5602957010269165, 0.6027089357376099, -0.03651220351457596, 0.12766164541244507, 0.44914957880973816, -0.2329598069190979, 0.07523620128631592, 0.1189463809132576, 0.3786359131336212, 0.357398122549057, -0.40323036909103394, 0.07234754413366318, -0.6680052876472473, 0.036301590502262115, -0.12135691940784454, -0.2471243143081665, 0.012553952634334564, 0.8008872270584106, -0.5521090030670166, 0.43521106243133545, 0.1320408284664154, -0.17120769619941711, -0.14077939093112946, -2.624800682067871, -0.2832529544830322, 0.516193270

In [ ]:
from pymongo import ReplaceOne

# Update the collection with the embeddings
requests = []
for doc, embedding in zip(docs, embeddings):
	doc[EMBEDDING_FIELD_NAME] = embedding
	requests.append(ReplaceOne({'_id': doc['_id']}, doc))
collection.bulk_write(requests)

# Step 4: Create a vector search index


In [ ]:
collection.create_search_index(
    {"definition":
        {"mappings": {"dynamic": True, "fields": {
            EMBEDDING_FIELD_NAME : {
                "dimensions": 512,
                "similarity": "dotProduct",
                "type": "knnVector"
                }}}},
     "name": ATLAS_VECTOR_SEARCH_INDEX_NAME
    }
)

'ImageSemanticSearch'

# Step 5: Query your data


In [ ]:
query = "Houses with swimming pools"
results = collection.aggregate([
    {
        '$search': {
            "index": ATLAS_VECTOR_SEARCH_INDEX_NAME,
            "knnBeta": {
                "vector": generate_embedding(query)[0],
                "k": 4,
                "path": EMBEDDING_FIELD_NAME}
        }
    }
])

results = list(results)
for document in results:
    print(f'{document["images"]},\n\n')

{'thumbnail_url': '', 'medium_url': '', 'picture_url': 'https://a0.muscache.com/im/pictures/bab17a39-035d-4ad8-ba38-e6bd9be4245f.jpg?aki_policy=large', 'xl_picture_url': ''},


{'thumbnail_url': '', 'medium_url': '', 'picture_url': 'https://a0.muscache.com/im/pictures/88710695-d25c-4144-a839-b93cfb2f4715.jpg?aki_policy=large', 'xl_picture_url': ''},


{'thumbnail_url': '', 'medium_url': '', 'picture_url': 'https://a0.muscache.com/im/pictures/25621df9-d04c-4c38-9055-f8cfb9771e13.jpg?aki_policy=large', 'xl_picture_url': ''},


{'thumbnail_url': '', 'medium_url': '', 'picture_url': 'https://a0.muscache.com/im/pictures/34569517/631c7e71_original.jpg?aki_policy=large', 'xl_picture_url': ''},




In [ ]:
from IPython.display import Image, display

for document in results:
    image_url = document['images']['picture_url']
    print(document['name'][:20])
    print(document['listing_url'])
    display(Image(url=image_url, width=400, height=400))


Greenwich Fun and Lu
https://www.airbnb.com/rooms/10459480


LUXURY HOUSE IN BARR
https://www.airbnb.com/rooms/2271702


Tranquil beach home 
https://www.airbnb.com/rooms/22198157


Private House Mindel
https://www.airbnb.com/rooms/2634829
